<a href="https://colab.research.google.com/github/JacobeCode/Neural_Network_For_Speech_Recognition/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install theano

# Main modules import
import keras
import librosa
import tensorflow
import os
import datetime

# Side modules import
import numpy as np
import theano as te
import matplotlib as plt
import pandas as pd

# Needed parts - import
from pathlib import Path
from google.colab import drive
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Flatten, LSTM, Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.utils.np_utils import to_categorical
from IPython.display import Audio, display
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, StratifiedKFold

# Root directory for files - for different directory - change here

# Python local version

# root_dir = Path.cwd()
# train_set_dir = os.path.join(Path.cwd(), "Recordings")
# train_set_list = os.listdir(train_set_dir)
# eval_set_dir = os.path.join(Path.cwd(), "Eval_Recordings")
# eval_set_list = os.listdir(eval_set_dir)

# Colab with Drive version
drive.mount('/content/gdrive/')

root_dir = '/content/gdrive/MyDrive/Speech_Recognition'
train_set_dir = '/content/gdrive/MyDrive/Speech_Recognition/Recordings'
train_set_list = os.listdir(train_set_dir)
eval_set_dir = '/content/gdrive/MyDrive/Speech_Recognition/Eval_Recordings'
eval_set_list = os.listdir(eval_set_dir)
os.chdir(root)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [8]:
# Parameters for comfortable changes later
n_mfcc = 20             # Number of MFCC coefficients
win_length = 512       # Length of MFCC window

In [33]:
# Data pre-processing - building database
train_set_base = pd.DataFrame()

os.chdir(train_set_dir)
MFCC_list = []
labels = []
for count, audio in enumerate(train_set_list):
    data, fs = librosa.load(audio, sr=None)
    data = data/max(np.abs(data))
    for iteration, item in enumerate(data):
        if item < 0.01:
            np.delete(data, iteration)
    MFCC = librosa.feature.mfcc(y=data, sr=fs, n_mfcc=n_mfcc, hop_length=int(win_length/2), win_length=win_length, n_fft=win_length)
    labels.append(audio[6])
    MFCC_list.append(MFCC)
max_len = max(np.shape(x)[1] for x in MFCC_list)
MFCC_temp = []
for item in MFCC_list:
    temp = []
    n_add = max_len - np.shape(item)[1]
    for coeff in item:
      temp.append(np.append(coeff, np.zeros(n_add)))
    MFCC_temp.append(temp)
    del temp
MFCC_list = MFCC_temp
del MFCC_temp

train_set_base["MFCC"] = MFCC_list
train_set_base["Labels"] = labels
os.chdir(root_dir)

# del MFCC_list
# del labels

In [30]:
# Data pre-processing - building database
eval_set_base = pd.DataFrame()

os.chdir(eval_set_dir)
MFCC_list = []
for count, audio in enumerate(eval_set_list):
    data, fs = librosa.load(audio, sr=None)
    data = data/max(np.abs(data))
    for iteration, item in enumerate(data):
        if np.abs(item) < 0.01:
            np.delete(data, iteration)
    MFCC = librosa.feature.mfcc(y=data, sr=fs, n_mfcc=n_mfcc, hop_length=int(win_length/2), win_length=win_length, n_fft=win_length)
    MFCC_list.append(MFCC)
max_len = max(np.shape(x)[1] for x in MFCC_list)
MFCC_temp = []
for item in MFCC_list:
    temp = []
    n_add = max_len - np.shape(item)[1]
    for coeff in item:
      temp.append(np.append(coeff, np.zeros(n_add)))
    MFCC_temp.append(temp)
    del temp
MFCC_list = MFCC_temp
del MFCC_temp

eval_set_base["MFCC"] = MFCC_list
os.chdir(root_dir)

# del MFCC_list

In [34]:
display(train_set_base)

,MFCC,Labels
0,"[[-662.9744262695312, -631.0436401367188, -568...",2
1,"[[-612.0942993164062, -573.1663818359375, -417...",0
2,"[[-571.8582763671875, -549.147216796875, -550....",1
3,"[[-595.4745483398438, -432.62554931640625, -26...",3
4,"[[-618.9461669921875, -591.8220825195312, -412...",9
...,...,...
149,"[[-483.2102966308594, -396.8646240234375, -362...",5
150,"[[-566.86279296875, -423.74737548828125, -267....",4
151,"[[-519.8472290039062, -470.52655029296875, -30...",3
152,"[[-559.2457275390625, -497.1453552246094, -432...",2


In [29]:
display(eval_set_base)

,MFCC
0,"[[-437.9372253417969, -349.28253173828125, -30..."
1,"[[-482.95367431640625, -488.2222595214844, -44..."
2,"[[-415.05047607421875, -408.4124755859375, -31..."
3,"[[-487.6944274902344, -455.0632629394531, -414..."
4,"[[-522.9053955078125, -283.26007080078125, -30..."
...,...
655,"[[-313.4111633300781, -316.08966064453125, -35..."
656,"[[-433.74310302734375, -353.6376647949219, -30..."
657,"[[-644.9763793945312, -635.8968505859375, -634..."
658,"[[-609.5606689453125, -601.455322265625, -310...."
